In [259]:
import tensorflow as tf
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import json
import copy

from plotly.subplots import make_subplots

In [260]:
class HyperTune:
    def __init__(self, val_score_key: str):

        self._data = pd.DataFrame({
            'Model':            [],
            'SGD 0.0001':       [],
            'SGD 0.001':        [],
            'SGD 0.01':         [],
            'Adagrad 0.0001':   [],
            'Adagrad 0.001':    [],
            'Adagrad 0.01':     [],
            'Adam 0.0001':      [],
            'Adam 0.001':       [],
            'Adam 0.01':        []
        })

        self._val_score_key = val_score_key

    def add_data(self, history: dict):
        model_performance = {}

        for h in list(history.keys()):
            if(h != 'Model'):
                model_performance[h] = {self._val_score_key: max(history[h][self._val_score_key]), 'val_loss': min(history[h]['val_loss'])}
            else:
                model_performance[h] = history[h]

        self._data = pd.concat([self._data, pd.DataFrame(model_performance)])
    
    def get_table(self):
        new_table = copy.copy(self._data)

        return new_table.set_index(['Model', new_table.index])
    
    def get_overview(self):
        new_frame = pd.DataFrame({
            self._val_score_key: np.sum(self._data.T.loc[slice(None), [self._val_score_key]], axis = 1),
            'val_loss': np.sum(self._data.T.loc[slice(None), ['val_loss']], axis = 1)
            })
        
        new_frame = new_frame.drop('Model', axis = 0)

        new_frame = np.divide(new_frame, self._data.shape[0] / 2)
        
        return new_frame.T

In [261]:
def plot_training_performance(history: dict, train_accuracy_key:str, val_accuracy_key: str):

    epochs = [len(history[i][train_accuracy_key]) for i in list(history.keys())]
    x = np.arange(1, max(epochs))

    fig = make_subplots(specs = [[{'secondary_y': True}]])

    for h in list(history.keys()):
        fig.add_trace(go.Scatter(
            x = x,
            y = history[h][train_accuracy_key],
            mode = 'lines',
            name = f'{h}: {train_accuracy_key}'
        ),
        secondary_y = False)

        fig.add_trace(go.Scatter(
            x = x,
            y = history[h]['loss'],
            mode = 'lines',
            name = f'{h}: training_loss'
        ),
        secondary_y = True)

        fig.add_trace(go.Scatter(
            x = x,
            y = history[h][val_accuracy_key],
            mode = 'lines',
            name = f'{h}: {val_accuracy_key}'
        ),
        secondary_y = False)

        fig.add_trace(go.Scatter(
            x = x,
            y = history[h]['val_loss'],
            mode = 'lines',
            name = f'{h}: validation_loss'
        ),
        secondary_y = True)

    fig.update_layout(
        title = 'Training Performance',
        xaxis_title = 'Epoch'
    )

    fig.update_yaxes(title_text = 'Accuracy', secondary_y = False)
    fig.update_yaxes(title_text = 'Loss', secondary_y = True)

    fig.show()

In [262]:
classification_tune = HyperTune('val_categorical_accuracy')
regression_tune = HyperTune('val_root_mean_squared_error')

1. Real-data only

In [263]:
files = {
    'SGD 0.0001': "models/RS_SGD_0.0001_pointnet_history.json", 
    'SGD 0.001': "models/RS_SGD_0.001_pointnet_history.json", 
    'SGD 0.01': "models/RS_SGD_0.01_pointnet_history.json",
    'Adagrad 0.0001': "models/RS_Adagrad_0.0001_pointnet_history.json", 
    'Adagrad 0.001': "models/RS_Adagrad_0.001_pointnet_history.json", 
    'Adagrad 0.01': "models/RS_Adagrad_0.01_pointnet_history.json",
    'Adam 0.0001': "models/RS_Adam_0.0001_pointnet_history.json", 
    'Adam 0.001': "models/RS_Adam_0.001_pointnet_history.json", 
    'Adam 0.01': "models/RS_Adam_0.01_pointnet_history.json",
    }

histories = {}
for key in files.keys():
    with open(files[key], 'r') as f:
        histories[key] = json.load(f)

plot_training_performance(histories, 'categorical_accuracy', 'val_categorical_accuracy')

histories['Model'] = 'RS'
classification_tune.add_data(histories)

FileNotFoundError: [Errno 2] No such file or directory: 'models/RS_SGD_0.0001_pointnet_history.json'

2. Simulated-data only

In [ ]:
files = {
    'SGD 0.0001': "models/SIM_SGD_0.0001_pointnet_history.json", 
    'SGD 0.001': "models/SIM_SGD_0.001_pointnet_history.json", 
    'SGD 0.01': "models/SIM_SGD_0.01_pointnet_history.json",
    'Adagrad 0.0001': "models/SIM_Adagrad_0.0001_pointnet_history.json", 
    'Adagrad 0.001': "models/SIM_Adagrad_0.001_pointnet_history.json", 
    'Adagrad 0.01': "models/SIM_Adagrad_0.01_pointnet_history.json",
    'Adam 0.0001': "models/SIM_Adam_0.0001_pointnet_history.json", 
    'Adam 0.001': "models/SIM_Adam_0.001_pointnet_history.json", 
    'Adam 0.01': "models/SIM_Adam_0.01_pointnet_history.json",
    }

histories = {}
for key in files.keys():
    with open(files[key], 'r') as f:
        histories[key] = json.load(f)

plot_training_performance(histories, 'categorical_accuracy', 'val_categorical_accuracy')

histories['Model'] = 'SIM'
classification_tune.add_data(histories)

3. Real-data only (partial sets)

In [ ]:
for rs_set in ['RS80', 'RS60', 'RS40', 'RS20']:
    files = {
        'SGD 0.0001': f"models/{rs_set}_SGD_0.0001_pointnet_history.json", 
        'SGD 0.001': f"models/{rs_set}_SGD_0.001_pointnet_history.json", 
        'SGD 0.01': f"models/{rs_set}_SGD_0.01_pointnet_history.json",
        'Adagrad 0.0001': f"models/{rs_set}_Adagrad_0.0001_pointnet_history.json", 
        'Adagrad 0.001': f"models/{rs_set}_Adagrad_0.001_pointnet_history.json", 
        'Adagrad 0.01': f"models/{rs_set}_Adagrad_0.01_pointnet_history.json",
        'Adam 0.0001': f"models/{rs_set}_Adam_0.0001_pointnet_history.json", 
        'Adam 0.001': f"models/{rs_set}_Adam_0.001_pointnet_history.json", 
        'Adam 0.01': f"models/{rs_set}_Adam_0.01_pointnet_history.json",
        }

    histories = {}
    for key in files.keys():
        with open(files[key], 'r') as f:
            histories[key] = json.load(f)

    plot_training_performance(histories, 'categorical_accuracy', 'val_categorical_accuracy')

    histories['Model'] = rs_set
    classification_tune.add_data(histories)

4. Augmented data

In [ ]:
for ag_set in ['AUG80', 'AUG60', 'AUG40', 'AUG20']:
    files = {
        'SGD 0.0001': f"models/{ag_set}_SGD_0.0001_pointnet_history.json", 
        'SGD 0.001': f"models/{ag_set}_SGD_0.001_pointnet_history.json", 
        'SGD 0.01': f"models/{ag_set}_SGD_0.01_pointnet_history.json",
        'Adagrad 0.0001': f"models/{ag_set}_Adagrad_0.0001_pointnet_history.json", 
        'Adagrad 0.001': f"models/{ag_set}_Adagrad_0.001_pointnet_history.json", 
        'Adagrad 0.01': f"models/{ag_set}_Adagrad_0.01_pointnet_history.json",
        'Adam 0.0001': f"models/{ag_set}_Adam_0.0001_pointnet_history.json", 
        'Adam 0.001': f"models/{ag_set}_Adam_0.001_pointnet_history.json", 
        'Adam 0.01': f"models/{ag_set}_Adam_0.01_pointnet_history.json",
        }

    histories = {}
    for key in files.keys():
        with open(files[key], 'r') as f:
            histories[key] = json.load(f)

    plot_training_performance(histories, 'categorical_accuracy', 'val_categorical_accuracy')

    histories['Model'] = ag_set
    classification_tune.add_data(histories)

In [ ]:
for ag_set in ['RS_PT']:
    files = {
        'SGD 0.0001': f"models/{ag_set}_SGD_0.0001_pointnet_history.json", 
        'SGD 0.001': f"models/{ag_set}_SGD_0.001_pointnet_history.json", 
        'SGD 0.01': f"models/{ag_set}_SGD_0.01_pointnet_history.json",
        'Adagrad 0.0001': f"models/{ag_set}_Adagrad_0.0001_pointnet_history.json", 
        'Adagrad 0.001': f"models/{ag_set}_Adagrad_0.001_pointnet_history.json", 
        'Adagrad 0.01': f"models/{ag_set}_Adagrad_0.01_pointnet_history.json",
        'Adam 0.0001': f"models/{ag_set}_Adam_0.0001_pointnet_history.json", 
        'Adam 0.001': f"models/{ag_set}_Adam_0.001_pointnet_history.json", 
        'Adam 0.01': f"models/{ag_set}_Adam_0.01_pointnet_history.json",
        }

    histories = {}
    for key in files.keys():
        with open(files[key], 'r') as f:
            histories[key] = json.load(f)

    plot_training_performance(histories, 'categorical_accuracy', 'val_categorical_accuracy')

    histories['Model'] = ag_set

In [ ]:
for ag_set in ['RS_PT']:
    files = {
        'SGD 0.0001': f"models/{ag_set}_SGD_0.0001_tnet_history.json", 
        'SGD 0.001': f"models/{ag_set}_SGD_0.001_tnet_history.json", 
        'SGD 0.01': f"models/{ag_set}_SGD_0.01_tnet_history.json",
        'Adagrad 0.0001': f"models/{ag_set}_Adagrad_0.0001_tnet_history.json", 
        'Adagrad 0.001': f"models/{ag_set}_Adagrad_0.001_tnet_history.json", 
        'Adagrad 0.01': f"models/{ag_set}_Adagrad_0.01_tnet_history.json",
        'Adam 0.0001': f"models/{ag_set}_Adam_0.0001_tnet_history.json", 
        'Adam 0.001': f"models/{ag_set}_Adam_0.001_tnet_history.json", 
        'Adam 0.01': f"models/{ag_set}_Adam_0.01_tnet_history.json",
        }

    histories = {}
    for key in files.keys():
        with open(files[key], 'r') as f:
            histories[key] = json.load(f)

    plot_training_performance(histories, 'root_mean_squared_error', 'val_root_mean_squared_error')

    histories['Model'] = ag_set
    regression_tune.add_data(histories)

In [ ]:
with open('figures/classification_tune_overview.tex', 'w') as tf:
    table_out = classification_tune.get_overview()

    new_index = {}
    for i in list(table_out.T.columns):
        new_index[i] = i.replace('_', ' ')

    table_out = table_out.T.rename(columns = new_index)
    tf.write(table_out.to_latex(float_format = "%0.3f"))

with open('figures/regression_tune_overview.tex', 'w') as tf:
    table_out = regression_tune.get_overview()

    new_index = {}
    for i in list(table_out.T.columns):
        new_index[i] = i.replace('_', ' ')

    table_out = table_out.T.rename(columns = new_index)
    tf.write(table_out.to_latex(float_format = "%0.3f"))

In [ ]:
classification_tune.get_table()

SGD 0.0001  SGD 0.001  SGD 0.01  \
Model                                                             
RS    val_categorical_accuracy    0.851852   0.848485  0.868687   
      val_loss                    0.419371   0.452852  0.354366   
SIM   val_categorical_accuracy    0.896667   0.959583  0.952500   
      val_loss                    0.420158   0.195056  0.195816   
RS80  val_categorical_accuracy    0.888889   0.845118  0.858586   
      val_loss                    0.309323   0.390523  0.390002   
RS60  val_categorical_accuracy    0.872054   0.865320  0.895623   
      val_loss                    0.359704   0.365537  0.297233   
RS40  val_categorical_accuracy    0.875421   0.885522  0.902357   
      val_loss                    0.330269   0.311266  0.303387   
RS20  val_categorical_accuracy    0.865320   0.861953  0.872054   
      val_loss                    0.367085   0.361307  0.342375   
AUG80 val_categorical_accuracy    0.883265   0.908770  0.922111   
      val_loss                    0.397981   0.273288  0.197314   
AUG60 val_categorical_accuracy    0.879341   0.911124  0.883265   
      val_loss                    0.324805   0.288233  0.350564   
AUG40 val_categorical_accuracy    0.900726   0.890328  0.917599   
      val_loss                    0.253918   0.513766  0.210895   
AUG20 val_categorical_accuracy    0.898568   0.915637  0.912498   
      val_loss                    0.290682   0.239180  0.231667   

                                Adagrad 0.0001  Adagrad 0.001  Adagrad 0.01  \
Model                                                                         
RS    val_categorical_accuracy        0.868687       0.861953      0.875421   
      val_loss                        0.391816       0.449106      0.396603   
SIM   val_categorical_accuracy        0.920833       0.933958      0.901250   
      val_loss                        0.253423       0.268993      0.549241   
RS80  val_categorical_accuracy        0.898990       0.885522      0.909091   
      val_loss                        0.285869       0.357136      0.290027   
RS60  val_categorical_accuracy        0.875421       0.868687      0.888889   
      val_loss                        0.341766       0.391359      0.345685   
RS40  val_categorical_accuracy        0.865320       0.875421      0.885522   
      val_loss                        0.347659       0.374895      0.312723   
RS20  val_categorical_accuracy        0.841751       0.872054      0.872054   
      val_loss                        0.414066       0.377021      0.357921   
AUG80 val_categorical_accuracy        0.881303       0.899549      0.901511   
      val_loss                        0.386606       0.372256      0.282306   
AUG60 val_categorical_accuracy        0.900726       0.891112      0.902492   
      val_loss                        0.333841       0.364834      0.232476   
AUG40 val_categorical_accuracy        0.877771       0.887385      0.913675   
      val_loss                        0.377198       0.362212      0.264963   
AUG20 val_categorical_accuracy        0.893663       0.874436      0.919953   
      val_loss                        0.325637       0.428122      0.203846   

                                Adam 0.0001  Adam 0.001  Adam 0.01  
Model                                                               
RS    val_categorical_accuracy     0.858586    0.892256   0.781145  
      val_loss                     0.445519    0.289557   0.591512  
SIM   val_categorical_accuracy     0.914792    0.868542   0.853542  
      val_loss                     0.229981    0.694730   0.814828  
RS80  val_categorical_accuracy     0.888889    0.882155   0.824916  
      val_loss                     0.331552    0.317196   0.481583  
RS60  val_categorical_accuracy     0.865320    0.939394   0.740741  
      val_loss                     0.463110    0.162279   0.584929  
RS40  val_categorical_accuracy     0.872054    0.888889   0.875421  
      val_loss                     0.374583    0.311096   0